In [3]:
import numpy as np
import matplotlib.pyplot as plt
import options_trading as opt
%load_ext autoreload
%autoreload 2

### Price a Call Spread

In [4]:
def call_spread (S0, K1, K2, T, r, vol):
    C1=opt.black_scholes(r, S0, K1, T, vol, type="c")
    C2=opt.black_scholes(r, S0, K2, T, vol, type="c")
    return C1-C2

In [6]:
call_spread(100, 86, 90, 1, 0.04, 0.2)

2.958252649707731

In [7]:
S0=100
K=100
T=1
H=125
r=0.06
N=3
u=1.1
d=1/u
dt=T/N
opttype='C'

In [8]:
def binomial_tree_slow (S0, K, T, H, r, N, u, opttype='C'):
    dt=T/N
    d=1/u
    q=(np.exp(-r*dt)-u)/(u-d)

    #Compute the final prices at maturity
    S=np.zeros(N+1)
    for j in range (N+1):
        S[j]=S0*u**j*d**(N-j)
    
    #Compute option payoff at maturity
    C=np.zeros(N+1)
    for j in range (N+1):
        if opttype=='C':
            C[j]=max(0,S[j]-K)
        else:
            C[j]=max(0,K-S[j])
    
    for j in range (N+1):
        if S[j]>=H:
            C[j]=0
            
    #work backward
    for i in np.arange(N-1, -1, -1):
        for j in range (i+1):
            S[j]=S0*u**j*d**(i-j)
            if S[j]>H:
                C[j]=0
            else:
                C[j]=np.exp(-r*dt)*(q*C[j+1]+(1-q)*C[j])
    return C[0]

In [9]:
#Fast implementation of binomial tree

def binomial_tree_fast (S0, K, T, H, r, N, u, opttype='C'):
    dt=T/N
    d=1/u
    q=(np.exp(-r*dt)-u)/(u-d)
    
    S=S0*d**np.arange(N,-1,-1)*u**np.arange(0,N+1,1)

    if opttype=='C':
        C=np.maximum(S-K,0)
    else:
        C=np.maximum(K-S,0)

    C[S>=H]=0

    for i in np.arange(N-1, -1, -1):
        S=S0*d**np.arange(i,-1,-1)*u**np.arange(0,i+1,1)
        C[:i+1]=np.exp(-r*dt)*(q*C[1:i+2]+(1-q)*C[0:i+1])
        C = C[:-1]
        C[S>=H]=0
    
    return C[0]

In [10]:
binomial_tree_fast(100, 100, 1, 125, 0.06, 3, 1.1)

18.107680656489546

## Monte Carlo Process

In [11]:
S0=100
K=100
T=1
H=125
r=0.01
vol=0.2

N=100   #number of timesteps
M=1000  #number of simulations


def monte_carlo_pricing(S0, K, T, H, r, vol, N, M):
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        BARRIER=False
        St=S0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
        
            if St>=H:
                BARRIER=True
                break
        
        if BARRIER:
            CT=0
        else:
            CT=max(0,K-St)
            
        sum_CT=sum_CT+CT
        sum_CT2=sum_CT2+CT*CT
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    
    return (C0, SE)

In [12]:
monte_carlo_pricing(100, 100, 1, 125, 0.06, 0.2, 100, 1000)

(5.236074801556796, 0.27345184419499585)

### Price a Digit option

In [13]:
def digit_option(S0, K, T, r, vol, N, M, opttype='C'):
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        St=S0
        CT=0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
        
        if opttype=='C':
            if St>=K:
                CT=1
        
        else:
            if St<K:
                CT=1
    
        sum_CT=sum_CT+CT
        sum_CT2=sum_CT2+CT*CT
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    
    return C0

In [14]:
digit_option(100, 110, 1, 0.04, 0.2, 100, 1000)

0.3468449875339887

### Up-and-in barrier option

In [15]:
def up_and_in_call(S0, K, H, T, r, vol, N, M):
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        BARRIER=False
        St=S0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
        
            if St>=H:
                BARRIER=True
        
        if BARRIER:
            CT=max(0,St-K)
        else:
            CT=0
            
        sum_CT=sum_CT+CT
        sum_CT2=sum_CT2+CT*CT
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    
    return C0

In [16]:
up_and_in_call(130, 130, 150, 1, 0.04, 0.1, 100, 1000)

4.8157638728167225

### Up-and-out barrier option

In [17]:
def up_and_out_call(S0, K, H, T, r, vol, N, M, rebate=False, rebate_percent=0):
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        BARRIER=False
        St=S0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
        
            if St>=H:
                BARRIER=True
                break
        
        if BARRIER:
            if rebate:
                CT=rebate_percent*max(0,H-K)
            else:
                CT=0
        else:
            CT=max(0,St-K)
            
        sum_CT=sum_CT+CT
        sum_CT2=sum_CT2+CT*CT
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    
    return C0

In [18]:
up_and_out_call(130, 130, 150, 1, 0.04, 0.1, 100, 1000, rebate=True, rebate_percent=0.1)

3.78496557592553

In [19]:
up_and_out_call(130, 130, 150, 1, 0.04, 0.1, 100, 1000)

3.197928216315877

### Down-and-in barrier option

In [20]:
def down_and_in_put(S0, K, H, T, r, vol, N, M):
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        BARRIER=False
        St=S0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
        
            if St<=H:
                BARRIER=True
        
        if BARRIER:
            CT=max(0,K-St)
        else:
            CT=0
            
        sum_CT=sum_CT+CT
        sum_CT2=sum_CT2+CT*CT
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    
    return C0

In [21]:
down_and_in_put(130, 130, 100, 1, 0.04, 0.1, 100, 1000)

0.1274447280592388

### Down-and-out barrier option

In [22]:
def down_and_out_put(S0, K, H, T, r, vol, N, M):
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        BARRIER=False
        St=S0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
        
            if St<=H:
                BARRIER=True
                break
        
        if BARRIER:
            CT=0
        else:
            CT=max(0,K-St)
            
        sum_CT=sum_CT+CT
        sum_CT2=sum_CT2+CT*CT
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    
    return C0

### Price a Ladder option

In [23]:
def ladder_option(S0, strikes, barriers, T, r, vol, N, M, rebates):
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        St=S0
        rebate_total=0
        for level in range (len(strikes)):
            BARRIER=False
    
            for j in range (N):
                epsilon=np.random.normal()
                Stn=St*np.exp(nudt+volsdt*epsilon)
                St=Stn
        
                if level < len(barriers) and St>=barriers[level]:
                    BARRIER=True
                    break
        
            if BARRIER:
                rebate_total+=rebates[level]*max(0,barriers[level]-strikes[level])
                continue
                
            else:
                payoff=max(0,St-strikes[level])
                rebate_total+=payoff
            
        sum_CT+=rebate_total
        sum_CT2+=rebate_total**2
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    
    return C0

In [24]:
ladder_option(100, [100, 105, 110], [105, 110], 1, 0.04, 0.1, 100, 1000, [0.05, 0.05])

5.203286376030618

### Price a Lookback option

In [25]:
def fixed_strike_lookback_ladder(S0, K, T, r, vol, N, M, tick=0.01, rebate_value=0.01, opttype='C'):
    #Pricing lookback option with fixed strike as a ladder option
    dt = T / N
    nudt = (r - 0.5 * vol**2) * dt
    volsdt = vol * np.sqrt(dt)
    erdt = np.exp(-r * dt)

    # Création des strikes ladder
    strikes = np.arange(K, K + tick * (N + 1), tick)    
    rebates=(len(strikes)-1)*[rebate_value]
    
    sum_CT = 0
    sum_CT2 = 0
    
    for i in range(M):
        St = S0
        rebate_sum = 0
        
        for j in range(N):
            epsilon = np.random.normal()
            Stn = St * np.exp(nudt + volsdt * epsilon)
            St = Stn
            
            if opttype == 'C':  # Option Call
                for i in range(len(strikes) - 1):
                    if St >= strikes[i + 1]:
                        rebate_sum += rebates[i]
                        break
            else:  # Option Put
                for i in range(len(strikes) - 1):
                    if St <= strikes[i]:
                        rebate_sum += rebates[i]
                        break
        
        if opttype == 'C':
            CT = max(0, St - K) + rebate_sum
        else:
            CT = max(0, K - St) + rebate_sum
        
        sum_CT += CT
        sum_CT2 += CT**2
    
    C0 = np.exp(-r * T) * sum_CT / M
    return C0

In [28]:
fixed_strike_lookback_ladder(100, 130, 1, 0.04, 0.2, 100, 10000, tick=0.01, rebate_value=0.01, opttype='C')

1.6381070442632737

In [29]:
def fixed_strike_lookback(S0, K, T, r, vol, N, M, opttype='C'):
    #Pricing lookback optionwith fixed strike using Monte Carlo process
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        max_price=S0
        min_price=S0
        St=S0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
            max_price=max(max_price, St)
            min_price=min(min_price, St)
                
        if opttype=='C':     #it's a call option
            CT=max(0,max_price-K)
        else:                #it's a put option
            CT=max(0,K-min_price)      
        
        sum_CT+=CT
        sum_CT2+=CT**2
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    
    return C0

In [30]:
fixed_strike_lookback(100, 130, 1, 0.04, 0.2, 100, 10000, opttype='C')

2.411799074777679

In [31]:
fixed_strike_lookback(100, 100, 1, 0.06, 0.2, 100, 10000, opttype='C')

18.405303808775663

In [32]:
def floating_strike_lookback(S0, T, r, vol, N, M, opttype='C'):
    #Pricing lookback option with floating strike using Monte Carlo process
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        K=S0
        St=S0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
            if opttype=='C':
                K=min(K, St)
            else:
                K=max(K, St)
                
        if opttype=='C':     #it's a call option
            CT=max(0,St-K)
        else:                #it's a put option
            CT=max(0,K-St)      
        
        sum_CT+=CT
        sum_CT2+=CT**2
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    
    return C0

In [33]:
floating_strike_lookback(100, 1, 0.04, 0.2, 100, 1000, opttype='C')

15.805300532911453

### Knock-in Reverse Convertible

In [73]:
def knock_in_reverse_convertible(A, S0, H, c, t, T, r, vol, N, M):     ##t in months
    ## Renvoie la valeur d'un coupon en utilisant c la valeur d'une obligation basique
    obs=T/(t/12)
    V=0
    final_discount=1/((1+c)**obs)
    for i in range (1,int(obs)+1):
        V+=1/((1+c)**i)

    P=down_and_in_put(S0, S0, H, T, r, vol, N, M)
    
    return ((1-final_discount+P/A)/V)*100     ##pour avoir le résultat en pourcentage

In [74]:
knock_in_reverse_convertible(100, 100, 80, 0.0175, 6, 2, 0.04, 0.2, 100, 1000)

3.177577929746367

### Autocallable

In [1]:
def autocallable (A, S0, H1, H2, H3, c, t, T, r, vol, N, M):
    ## H1 est la barrière basse de knock-in, H2 est la barrière pour les coupons
    ## et H3 est la barrière haute (de knock-out avec rebate)
        
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)
    obs=int(N/(t/12))

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        St=S0
        knocked_out=False
        knocked_in=False
        CT=0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
            
            ##Down and in put option à H1
            if St<H1:
                knocked_in=True

            if (j+1)%obs==0:

                if St>=H3:
                    knocked_out=True
                    break
                
                CT+=call_spread(S0, H2-c*100, H2, (T - (j+1) * dt), r, vol)
        
        if knocked_in:
            CT+= -max(0,S0-St)
        
        if knocked_out or St>H1:
            CT+=A    
        
        sum_CT+=CT
        sum_CT2+=CT**2
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    return C0

In [5]:
autocallable(100, 100, 80, 90, 110, 0.04, 6, 2, 0.02, 0.2, 100, 1000)

66.24523004863457

In [83]:
def autocallable_unconditional (A, S0, H1, H2, H3, c, t, T, r, vol, N, M):
    ## H1 est la barrière basse de knock-in, H2 est la barrière pour les coupons
    ## et H3 est la barrière haute (de knock-out avec rebate)
        
    dt=T/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)
    obs=int(N/(t/12))

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        St=S0
        knocked_out=False
        knocked_in=False
        CT=0
        total_coupon=0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
            
            ##Down and in put option à H1
            if St<H1:
                knocked_in=True

            if (j+1)%obs==0:
                total_coupon+=c

                if St>=H3:
                    knocked_out=True
                    CT=A+A*total_coupon
                    break
                
                CT+=call_spread(S0, H2-c*100, H2, (T - (j+1) * dt), r, vol)
        
        if knocked_in:
            CT+= -max(0,S0-St)
        
        if not knocked_out:
            CT+=A+A*total_coupon  
        
        sum_CT+=CT
        sum_CT2+=CT**2
    
    C0=np.exp(-r*T)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    return C0

In [84]:
autocallable_unconditional(100, 100, 80, 90, 110, 0.04, 6, 2, 0.02, 0.2, 100, 1000)

87.02052656930199

### Asian option

In [67]:
def asian_out_artihmetic_option(S0, K, T1, T2, r, vol, N, M):
    #Pricing Asian (arithmetic) out option
    dt=(T2-T1)/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        St=S0
        At_sum=0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
            At_sum+=St
            
        A=At_sum/N
        CT=np.max(A-K, 0)
        
        sum_CT+=CT
        sum_CT2+=CT**2
    
    C0=np.exp(-r*T2)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T2)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    
    return C0

In [68]:
asian_out_artihmetic_option(100, 0, 1, 0.04, 0.2, 100, 1000)

2.1092622790715576

In [69]:
#Antithetic technique : to improve accuracy
def asian_out_option_antithetic(S0, T1, T2, r, vol, N, M):
    #Pricing Asian (arithmetic) out option
    dt=(T2-T1)/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        St=S0
        At1_sum=0
        At2_sum=0
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn1=St*np.exp(nudt+volsdt*epsilon)
            Stn2=St*np.exp(nudt-volsdt*epsilon)
            St1=Stn1
            St2=Stn2
            At1_sum+=St1
            At2_sum+=St2
            
        A1=At1_sum/N
        A2=At2_sum/N
        CT=(np.max(A1-K, 0)+np.max(A2-K, 0))/2
        
        sum_CT+=CT
        sum_CT2+=CT**2
    
    C0=np.exp(-r*T2)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T2)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    
    return C0

In [70]:
asian_out_option_antithetic(100, 0, 1, 0.04, 0.2, 100, 1000)

0.03849506586989446

In [71]:
def asian_out_geometric_option(S0, K, T1, T2, r, vol, N, M):
    #Pricing Asian (geometric) out option
    dt=(T2-T1)/N
    nudt=(r-0.5*vol**2)*dt
    volsdt=vol*np.sqrt(dt)
    erdt=np.exp(-r*dt)

    sum_CT=0
    sum_CT2=0

    for i in range(M):
        St=S0
        GT=1
    
        for j in range (N):
            epsilon=np.random.normal()
            Stn=St*np.exp(nudt+volsdt*epsilon)
            St=Stn
            GT=GT*St
            
        G=GT**(1/N)
        CT=np.max(G-K, 0)
        
        sum_CT+=CT
        sum_CT2+=CT**2
    
    C0=np.exp(-r*T2)*sum_CT/M
    sigma=np.sqrt((sum_CT2-sum_CT*sum_CT/M)*np.exp(-2*r*T2)/(M-1))
    SE=sigma/np.sqrt(M)    #Compute the standard error
    
    return C0

In [72]:
asian_out_geometric_option(100, 0, 1, 0.04, 0.2, 100, 1000)

2.4645958400165267